Heavily inspired by the lesson 
https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/2/document-loading
I want to:
1. Get the contents of a git repo with lesson material (in github-pages).
2. Add the files with content (for now only the markdown files) and create a vector store.
3. Be able to chat with the material, ask questions like: 'Tell me about the concept blablathisnthat ...', or 'Ask a question about ...'. 

In [14]:
!pip install python-dotenv
!pip install pypdf 
!pip install markdown
!pip install openai
!pip install GitPython
!pip list

2452.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2459.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2466.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2473.40s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2480.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2487.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Package                 Version
----------------------- -----------
aiohttp                 3.8.5
aiosignal               1.3.1
asttokens               2.2.1
async-timeout           4.0.3
attrs                   23.1.0
backcall                0.2.0
certifi                 2023.7.22
charset-normalizer      3.2.0
comm                    0.1.4
dataclasses-json        0.5.14
debugpy                 1.6.7.post1
decorator               5.1.1
executing               1.2.0
frozenlist              1.4.0
gitdb                   4.0.10
GitPython               3.1.32
greenlet                2.0.2
idna                    3.4
ipykernel               6.25.1
ipython                 8.14.0
jedi                    0.19.0
jupyter_client          8.3.0
jupyter_core            5.3.1
langchain               0.0.263
langsmith               0.0.22
Markdown                3.4.4
marshmallow             3.20.1
matplotlib-inline       0.1.6
multidict               6.0.4
mypy-extensions         1.0.0
nest-asyncio 

In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.document_loaders import PyPDFLoader
#loader = PyPDFLoader("samples/tutorial_Class.pdf")
loader = PyPDFLoader("samples/tutorial_Class.pdf")
pages = loader.load()

In [4]:
pages[0]

[Document(page_content='Stéphane Ducasse, Lukas Renggli, David C. Shaffer, Rick Zacconewith Michael DaviesDynamic WebDevelopmentwith', metadata={'source': 'samples/tutorial_Class.pdf', 'page': 0}),
 Document(page_content='ii', metadata={'source': 'samples/tutorial_Class.pdf', 'page': 1}),
 Document(page_content='Dynamic Web Development with Seaside\nStephane Ducasse Lukas Renggli C. David Shaffer\nRick Zaccone\nwith Michael Davies\n16 July 2014', metadata={'source': 'samples/tutorial_Class.pdf', 'page': 2}),
 Document(page_content='ii\nThis book is available as a download from book.seaside.st.\nCopyright © July 16, 2014 Stéphane Ducasse, Lukas Renggli, C. David Shaffer, Rick Zaccone.\nThis book is licensed under a Creative Commons Attribution-Noncommercial-Share Alike 3.0\nlicense.\nPublished by Square Bracket Associates, Switzerland.\nsquarebracketassociates.org\nISBN 978-3-9523341-1-9\nFirst Edition, August, 2010.\nCover art by Samuel Morello.', metadata={'source': 'samples/tutorial_